In [1]:
import argparse
import collections
import torch
import numpy as np
import data_loader.data_loaders as module_data
import data_loader.preprocess as module_preprocess
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer


# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [2]:
import json
config = ConfigParser(json.load(open('config.json', 'r')))

In [3]:
logger = config.get_logger('train')
# preprocess
# config.init_obj('preprocess', module_preprocess)

# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired in the 0.8.0 release 

In [39]:
data_loader.TEXT.vocab.itos[:5]

['<unk>', '<pad>', '<START>', '<END>', '.']

In [25]:
from torchtext.data import Iterator

In [26]:
train_iter = Iterator(data_loader.dataset, 32)

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [35]:
val_iter = Iterator(valid_data_loader.dataset, 32)

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [6]:
for idx, i in enumerate(data_loader.data_iter):
    if idx == 5:
        break
    print(i.talk[0], i.talk[1])
    
#     print(i.response)

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  43, 1527,   27,  ...,   21,  199,   17],
        [2261,    5,   80,  ...,   17,    5,    6],
        ...,
        [   1,    1,   85,  ...,    1,    1,    1],
        [   1,    1,    4,  ...,    1,    1,    1],
        [   1,    1,    3,  ...,    1,    1,    1]]) tensor([ 6, 10, 32,  8, 20,  7, 32,  7, 22,  8, 12,  5,  8,  6,  8, 16, 27, 11,
         6,  7,  4, 14, 24, 28, 18, 12, 12, 14, 16,  4, 11,  7, 29, 32,  7, 12,
        11, 10,  8,  5,  5,  6,  4, 13, 12, 13, 16,  6,  7,  9,  5,  7, 13,  6,
        20, 12,  8, 11,  8, 25,  8, 10, 15, 12,  4, 24,  9, 17,  8, 30, 32,  9,
         5, 29,  7, 10, 20,  7,  8,  9,  8,  6, 30,  3,  8, 18,  8, 31, 13, 16,
         8, 32, 21,  4,  8, 12, 25, 32, 12, 14,  9, 25, 32, 23,  8,  8,  6, 25,
        17,  4, 32,  4, 23, 20, 10, 16,  7, 17,  6, 32,  9, 31, 11, 10, 20,  9,
        15,  9])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   7,  283,   64,  ...,   65,   38, 

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [41]:
print(config.__dict__)

{'_config': {'name': 'Chatbot', 'n_gpu': 1, 'arch': {'type': 'ChatbotModel', 'args': {}}, 'preprocess': {'type': 'ChatbotDataPreprocess', 'args': {'data_dir': 'data/cornell movie-dialogs corpus'}}, 'data_loader': {'type': 'ChatbotDataLoader', 'args': {'data_dir': 'data/cornell movie-dialogs corpus', 'filename': 'formatted_movie_lines.csv', 'batch_size': 128, 'sent_len': 256, 'init_token': '<START>', 'eos_token': '<END>', 'min_freq': 5, 'shuffle': True, 'validation_split': 0.1, 'num_workers': 2}}, 'optimizer': {'type': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 0, 'amsgrad': True}}, 'loss': 'nll_loss', 'metrics': ['accuracy', 'top_k_acc'], 'lr_scheduler': {'type': 'StepLR', 'args': {'step_size': 50, 'gamma': 0.1}}, 'trainer': {'epochs': 10, 'save_dir': 'saved/', 'save_period': 1, 'verbosity': 2, 'monitor': 'min val_loss', 'early_stop': 2, 'tensorboard': True}}, 'resume': None, '_save_dir': PosixPath('saved/models/Chatbot/1013_210419'), '_log_dir': PosixPath('saved/log/Chatbot/1013_21

In [45]:
import torch
import dill

In [48]:
with open("TEXT.Field","wb")as f:
     dill.dump(data_loader.TEXT, f)
# torch.save(data_loader.TEXT, 'text.dill', pickle_module=dill)

TypeError: 'generator' object is not callable

In [49]:
print(data_loader.TEXT)

In [54]:
from torchtext.data import Field

In [55]:
question_field = Field(sequential=True)

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [56]:
torch.save(question_field, 'question.Field')

In [57]:
import pickle

In [58]:
pickle.dump(data_loader.TEXT.vocab, open('test.vocab', 'wb'))

In [59]:
torch.save(data_loader.TEXT.vocab, 'test.vocab')

TypeError: 'generator' object is not callable